In [ ]:
%run base.ipynb

In [ ]:
experiment = 'few-shot-seeds-mulneg'
experiment = 'more-few-shot-seeds'
# experiment = 'more-batch64-few-shot-seeds'
# experiment = 'more-epochs-few-shot-seeds'
# experiment = 'moreepochs5-few-shot-seeds'
# experiment = 'more-few-shot-all-meaningless-seeds'
# experiment = 'templates0.1-few-shot-seeds'
# experiment = 'templates0.2-few-shot-seeds'
# experiment = 'templates0.2-seeds'

with_samples = False
# with_samples = True

results_path = get_experiment_path(experiment)

In [ ]:
results_path

In [ ]:
# model = 'mpnet'
model = 'all-mpnet'
# model = 'all-mini'
extra = ''
# extra = 'epoch2'
# extra = 'epoch3'
# extra = 'epoch5'

In [ ]:
import pandas as pd
pd.options.display.max_rows = None

In [ ]:
import pandas as pd
import numpy as np
import re
from tqdm import tqdm

def format_f(x):
    return re.sub('^0\.', '.', f'{x:.4f}')

regex = f'{model}.*{extra}-?([a-zA-Z_]*)?(?:-n(\d+)in(\d+))?-seed(\d+)'
regex = f'{model}.*{extra}-([a-zA-Z_]*)?(?:-n(\d+)in(\d+))?-seed(\d+)'

def read_results(name, templates=[]):
    extra_cols = ['per_map_size', 'map_size'] if 'few-shot' in experiment else []
    
    folder = f'{model}*{extra}*' if extra else f'{model}*'
    paths = list(results_path.glob(f'{folder}/results/{name}.json'))
    dfs = []
    for path in tqdm(paths):
#         print(path)
        df = pd.read_json(path, orient='index')
        if len(df.columns) == 1:
            df = pd.DataFrame([df.to_dict()[0]], columns=df.to_dict()[0].keys())
#         df[['model','template','per_map_size', 'map_size', 'seed']] = re.findall(r'([a-zA-Z]+-[a-zA-Z]+)-([a-zA-Z_]*)-n(\d+)in(\d+).*seed(\d+)', str(path))
#         df[['template','per_map_size', 'map_size', 'seed']] = re.findall(f'-([a-zA-Z_]*)-n(\d+)in(\d+).*seed(\d+)', str(path).replace('all-meaningless', 'all_meaningless'))
        df[['template','per_map_size', 'map_size', 'seed']] = re.findall(regex, str(path).replace('all-meaningless', 'all_meaningless').replace('all-all', 'all_all'))
#         print(df)
        df[extra_cols+['seed']] = df[extra_cols+['seed']].apply(lambda x: int(x))

#         display(df)
        dfs.append(df)
    df = pd.concat(dfs, ignore_index=True)
    df.loc[df['template']=='mulneg', 'template']='none'
    df.loc[df['template']=='', 'template']='none'
    ordered_templates = ['none', 'beginning', 'foo', 'pro_con', 'combined', 'all', 'all_meaningless']
    ordered_templates = ['none', 'beginning', 'pro_con', 'all', 'all_meaningless', 'all_all']
#     df = df.sort_values(by=['template'], key=lambda x: x.map({k:v for k,v in zip(ordered_templates, list(range(len(ordered_templates))))}))
#     display(df)
    df = pd.pivot_table(df, index=extra_cols+['template'], values=['p1', 'p5', 'mrr'],
                                aggfunc=[np.mean, np.std, np.count_nonzero], dropna=False)
    df = df[df['count_nonzero']['p1']==5]
    # display(df)

    formatter = lambda x: (x[0] if type(x[0])==str else format_f(x[0])+(f'\u00B1{format_f(x[1])}' if not pd.isna(x[1]) else '').replace('%',''))
    f_df = pd.DataFrame({col: zip(df['mean'][col], df['std'][col]) for col in df['mean'].columns})
    f_df.index = df.index
    f_df = f_df.applymap(formatter)
    
    f_df = f_df[['p1', 'p5', 'mrr']]
#     f_df.reset_index(level=0, inplace=True)
#     f_df.columns = f_df.columns.droplevel(2)
    
#     if not 'epoch' in extra:
    if extra_cols:
        f_df = f_df.reindex(ordered_templates, level=2)
    else:
        f_df = f_df.reindex(ordered_templates)
    
    return f_df

## with/without samples results

In [ ]:
print(regex)
path = '/home/ij/data/delib-results/results/moreepochs5-few-shot-seeds/all-mpnet-mulneg-epoch5-n16in32-seed13/results/avg.json'
re.findall(regex, str(path).replace('all-meaningless', 'all_meaningless'))

In [ ]:
if with_samples:
    df = pd.concat([read_results('avg'), read_results('annotated_samples_metrics')], keys=['test', 'samples'], axis=1)
    # df.columns = results_df.columns.droplevel(0)
    # df.reset_index(level=0)
else:
    df = read_results('avg')

In [ ]:
df

In [ ]:
def to_latex(df):
#     df = df.reset_index(level=0)
#     df = df.reset_index(level=0)
#     display(df)
    
#     df['per_map_size x map_size'] = df.apply(lambda x: f"{x['per_map_size']} x {x['map_size']}", axis=1)
#     df = df.drop(['per_map_size', 'map_size'], axis=1)
    return (df
            .applymap(lambda x: '$'+'_{\pm'.join(x.split('\u00B1'))+'}$' if '\u00B1' in x else x)            
#             .style.highlight_max().to_latex(escape=False)
            .to_latex(escape=False)
            .replace('per_map_size', '\#nodes').replace('map_size', '\#maps'))

## Main Few-shot Results 

In [ ]:
import itertools
sizes = [8, 16, 32, 64]

summary_df = df.query("template in ['none', 'beginning', 'pro_con', 'all']")
summary_df = df.query("template in ['none', 'beginning', 'foo', 'pro_con', 'all', 'all_meaningless']")
if 'few-shot' in experiment:
    summary_df = pd.concat([summary_df.query(f'per_map_size == {x} and map_size == {x}') for x in sizes])
summary_df

In [ ]:
def print_with_latex_mathbf(s):
    print(s)
    print(s.replace(' $', ' $\mathbf{').replace('_{', '}_{'))

In [ ]:
print_with_latex_mathbf(to_latex(summary_df).replace('beginning', 'parent/child').replace('pro_con', 'pro/con'))

## Filtered Results (Foobar etc..)

In [ ]:
foo_df = df.query("template in ['beginning', 'foo']")
foo_df = df.query("template in ['all']")
# foo_df = df.query("template in ['none']")
# foo_df = df.query("template in ['all', 'all_meaningless']")
foo_df = pd.concat([foo_df.query(f'per_map_size == {x} and map_size == {x}') for x in sizes])
foo_df

In [ ]:
print(to_latex(foo_df).replace('beginning', 'parent/child').replace('foo', 'foo/bar').replace('all_meaningless', 'all meaningless'))

In [ ]:
print_with_latex_mathbf(to_latex(foo_df.reset_index(level=2, drop=True)))

## #nodes vs #maps 

In [ ]:
ordered_related_sizes = list(dict.fromkeys(itertools.chain.from_iterable([(x, y), (y, x)] for x, y in itertools.product(sizes, sizes) if x != y)))
related_df = df.query("template == 'none'")
related_df = pd.concat([related_df.query(f'per_map_size == {x} and map_size == {y}') for x, y in ordered_related_sizes])
related_df

In [ ]:
print(to_latex(related_df.reset_index(level=2, drop=True)))